In [1]:
import datetime, json, os, time
import pandas as pd

from azure.cognitiveservices.language.luis.runtime import LUISRuntimeClient
from msrest.authentication import CognitiveServicesCredentials

from ratelimit import limits, sleep_and_retry

## More details here: https://docs.microsoft.com/en-us/azure/cognitive-services/luis/luis-reference-regions
#  e.g. https://xxx.api.cognitive.microsoft.com/
LUIS_RUNTIME_ENDPOINT="<YOUR RUNTIME ENDPOINT HERE>"
LUIS_RUNTIME_KEY="<YOUR LUIS KEY HERE>"
LUIS_RUNTIME_APPID="<YOUR LUIS APP ID>"

# Load data into dataframe
df = pd.read_csv("example.csv",  index_col='id')

# view df from file
df.head()

,text
id,
1,the fingerprint reader on my phone no longer w...
2,I am unable to login to the application
3,this is an example where someone has many issu...


In [2]:
# Instantiate a LUIS runtime client
clientRuntime = LUISRuntimeClient(LUIS_RUNTIME_ENDPOINT, CognitiveServicesCredentials(LUIS_RUNTIME_KEY))

In [3]:
# Define functions for prediction and entity parsing from response

@sleep_and_retry
@limits(calls=5, period=1)
def predict(app_id, slot_name, query):
    request = { "query" : str(query) }
    # Note be sure to specify, using the slot_name parameter, whether your application is in staging or production.
    resp = clientRuntime.prediction.get_slot_prediction(app_id=app_id, slot_name=slot_name, prediction_request=request)
    return (resp)

def get_all_values(d):
    if isinstance(d, dict):
        for v in d.values():
            yield from get_all_values(v)
    elif isinstance(d, list):
        for v in d:
            yield from get_all_values(v)
    else:
        yield d 

def get_entities(resp):
    return ','.join(get_all_values(list(resp.prediction.entities.values())))


In [4]:
# Create new column with entities from LUIS
df['entities'] = df.apply(lambda row: get_entities(predict(LUIS_RUNTIME_APPID, 'production', row['text'])), axis=1)

In [5]:
# df with entities column
df.head()

,text,entities
id,,
1,the fingerprint reader on my phone no longer w...,fingerprint
2,I am unable to login to the application,login
3,this is an example where someone has many issu...,"fingerprint,login"
